## 설치

deepspeed 는 자체적인 버전과 transformers 와의 통합을 위한 버전을 모두 제공합니다. 물론 파이토치와의 호환성도 당얀히 있습니다.

여기서는 일단 딥스피드를 설치하고, 딥스피드를 이용해 torch 모델, 그리고 간단한 transformers 모델을 실행하는 방법을 설명합니다.

예제들은 공식 DeepspeedExample 레포지토리에서 발췌했습니다! https://github.com/deepspeedai/DeepSpeedExamples.git

In [1]:
!pip install deepspeed transformers[deepspeed]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached hjson-3.1.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached nvidia_ml_py-13.580.82-py3-none-any.whl.metadata (9.6 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.11.0-py3-none-any.whl (375 kB)
Using cached einops-0.8.1-py3-none-any.whl (64 kB)
Using cached hjson-3.1.0-py3-none-any.whl (54 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
Using cached nvidia_ml_py-13.580.82-py3-none-any.whl (49 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  DEPRECATION: Building 'deepspeed' using the legacy setup.py bdist_wheel mechanis

## 1. 간단한 deepspeed 설정해서 돌려보기

딥스피드의 설정은 json 파일로 만들어 넣어주게 됩니다. `ds_config.json` 파일을 만들어서 저장합니다.
```json
{
  "fp16": {
    "enabled": true,
    "loss_scale": 0,
    "loss_scale_window": 1000,
    "initial_scale_power": 16,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 0.001,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 3e-7
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 0.001,
      "warmup_num_steps": 500
    }
  },
  "zero_optimization": {
    "stage": 2,
    "allgather_partitions": true,
    "allgather_bucket_size": 2e8,
    "reduce_scatter": true,
    "reduce_bucket_size": 2e8,
    "contiguous_gradients": true,
    "overlap_comm": true
  },
  "train_micro_batch_size_per_gpu": 10
}
```

각각이 의미하는 바는 다음과 같습니다.
`"fp16"` : 16비트 부동소수점 연산을 사용할지 여부와 관련된 설정입니다.  
`"optimizer"` : 옵티마이저의 종류와 하이퍼파라미터를 설정합니다.  
`"scheduler"` : 학습률 스케줄러의 종류와 하이퍼파라미터를 설정합니다.  
`"zero_optimization"` : ZeRO 최적화 기법의 단계를 설정합니다. 여기서 `"stage": 2` 는 ZeRO-2 를 사용하는 것을 의미합니다. 
`"train_micro_batch_size_per_gpu"` : 각 GPU당 마이크로 배치 크기를 설정합니다.

주의할 점은 여기서 **optimizer** 설정이 pytorch 의 설정으로 이루어지거나, transformers의 train 메서드에서 설정하는 것과 다르다는 점입니다. 별도로 deepspeed 설정에서 옵티마이저를 지정해주어야 합니다.

더 다양한 config 설정하는 법을 보고 싶다면 https://www.deepspeed.ai/docs/config-json/ 를 참고해볼 수 있습니다. 

### 1.1. 토치에서 쓰기: `cifar/cifar10_deepspeed.py`

1. deepspeed를 import 합니다.
2. argument 에서 ds_config에서 세세하게 지정하지 않는 것들을 설정해줍니다. (예: epoch, batch size, 혹은 그 외 네트워크 설정이나 실험에 필요한거)
3. `deepspeed.add_config_arguments(parser)` 로 deepspeed 모델 엔진을 위한 인자를 추가합니다. 여기서 아까 작성해놓은 `ds_config.json` 파일을 지정해줄 수 있습니다.
4. 모델 초기화 시 `deepspeed.initialize()` 를 사용합니다. 여기서 모델을 지정해주고 그것에 대한 `model_engine` 인스턴스를 받습니다. 이거를 통해서 학습 루프를 돌리게 됩니다.



### 가끔 kill 을 안하고 종료하는경우

포트가 이미 사용중이라는 에러가 뜨는 경우가 있습니다.

```
torch.distributed.DistNetworkError: The server socket has failed to listen on any local network address. port: 29500, useIpv6: false, code: -98, name: EADDRINUSE, message: address already in use
```

이렇게 해당 포트번호로 사용중인 프로세스를 확인합니다.

```
(dist) hail@hail-4090-3way:~/HDD/dk/distributed-deep-learning/deepspeed/cifar$ lsof -i :29500
COMMAND       PID USER   FD   TYPE     DEVICE SIZE/OFF NODE NAME
pt_data_w 1629447 hail   41u  IPv6 1423925726      0t0  TCP localhost:49956->localhost:29500 (ESTABLISHED)
pt_data_w 1629448 hail   47u  IPv6 1423964615      0t0  TCP *:29500 (LISTEN)
pt_data_w 1629448 hail   48u  IPv6 1423964616      0t0  TCP localhost:43806->localhost:29500 (ESTABLISHED)
pt_data_w 1629448 hail   49u  IPv6 1423655935      0t0  TCP localhost:29500->localhost:43806 (ESTABLISHED)
pt_data_w 1629448 hail   50u  IPv6 1423927837      0t0  TCP localhost:29500->localhost:49956 (ESTABLISHED)
pt_data_w 1629449 hail   41u  IPv6 1423925726      0t0  TCP localhost:49956->localhost:29500 (ESTABLISHED)
pt_data_w 1629451 hail   47u  IPv6 1423964615      0t0  TCP *:29500 (LISTEN)
pt_data_w 1629451 hail   48u  IPv6 1423964616      0t0  TCP localhost:43806->localhost:29500 (ESTABLISHED)
pt_data_w 1629451 hail   49u  IPv6 1423655935      0t0  TCP localhost:29500->localhost:43806 (ESTABLISHED)
pt_data_w 1629451 hail   50u  IPv6 1423927837      0t0  TCP localhost:29500->localhost:49956 (ESTABLISHED)
```

찾아서 PID 제거

```
(dist) hail@hail-4090-3way:~/HDD/dk/distributed-deep-learning/deepspeed/cifar$ kill -9 1629448
(dist) hail@hail-4090-3way:~/HDD/dk/distributed-deep-learning/deepspeed/cifar$ kill -9 1629449
(dist) hail@hail-4090-3way:~/HDD/dk/distributed-deep-learning/deepspeed/cifar$ kill -9 1629451
(dist) hail@hail-4090-3way:~/HDD/dk/distributed-deep-learning/deepspeed/cifar$ kill -9 1629447
```

## 2. Pipeline Parallelism

![https://www.deepspeed.ai/assets/images/pipe-schedule.png](https://www.deepspeed.ai/assets/images/pipe-schedule.png)

DeepSpeed는 **그래디언트 누적(Gradient Accumulation)** 을 사용하여 파이프라인 병렬처리를 구현합니다.

- 하나의 훈련 배치는 여러 개의 **마이크로배치(micro-batch)** 로 분할됩니다.
- 이 마이크로배치들은 파이프라인 스테이지에서 병렬로 처리됩니다.
- 한 스테이지가 마이크로배치에 대한 Forward Pass(순전파)를 완료하면, 그 결과물(Activation)은 즉시 파이프라인의 다음 스테이지로 전달됩니다.
- 마찬가지로, 다음 스테이지가 Backward Pass(역전파)를 완료하면, 그래디언트가 이전 스테이지로 거꾸로 전달됩니다.
- 각 스테이지는 Backward Pass를 수행하며 그래디언트를 로컬에 누적합니다. 
- (파이프라인이 한 번 다 돈 뒤) 모든 데이터 병렬 그룹(DP group)은 동시에 그래디언트 All-Reduce를 수행합니다.
- 마지막으로 옵티마이저가 모델 가중치를 업데이트합니다.

위 그림은 8개의 마이크로배치를 사용하는 하이브리드(2-way DP + 2-stage PP) 훈련 예시입니다.
- GPU 0과 GPU 2는 파이프라인(PP)으로 묶여 Forward(F)와 Backward(B)를 번갈아 수행합니다.
- 그 후, 이들은 각자의 데이터 병렬(DP) 파트너인 GPU 1, GPU 3과 각각 All-Reduce(AR)를 수행합니다.
- 최종적으로 두 파이프라인 스테이지가 각자의 가중치를 업데이트(U)합니다.

파이프라인 병렬처리를 사용하려면 모델이 **레이어의 순차적인 나열(sequence of layers)** 로 표현되어야 합니다. PyTorch의 torch.nn.Sequential이 파이프라인 병렬화에 가장 적절하며, DeepSpeed는 이를 코드 수정 없이 바로 병렬화할 수 있습니다.

DeepSpeed는 PipelineModule이라는 래퍼(wrapper)를 제공합니다.

```python
# 기존 nn.Sequential 모델
net = nn.Sequential(
    nn.Linear(in_features, hidden_dim),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_dim, out_features)
)

# DeepSpeed PipelineModule로 변환
from deepspeed.pipe import PipelineModule
net = PipelineModule(layers=net, num_stages=2)
```

PipelineModule은 layers 인자로 받은 모델을 num_stages=2 (즉, 2개의 스테이지)로 분할하여 각각의 GPU에 할당합니다. 만약 2개보다 많은 GPU가 있다면, DeepSpeed는 자동으로 하이브리드 데이터 병렬처리(DP)도 함께 사용합니다.

참고: 총 GPU 개수는 파이프라인 스테이지 개수로 나누어 떨어져야 합니다.

### (예시) AlexNet을 파이프라인 모델로 변환하기

AlexNet은 여러 nn.Sequential 서브 모듈로 구성되어 있습니다. 이를 PipelineModule로 만들려면, 이 서브 모듈들을 하나의 평평한(flat) 레이어 시퀀스로 만들어 반환하는 헬퍼(helper) 함수를 추가하면 됩니다.

```python
# 기존 AlexNet 정의
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(...)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(...)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# DeepSpeed 파이프라인용으로 수정
class AlexNetPipe(AlexNet):
    def to_layers(self):
        # 모든 서브 모듈을 하나의 리스트로 펼칩니다.
        layers = [
            *self.features,
            self.avgpool,
            lambda x: torch.flatten(x, 1), # 일반 함수(lambda)도 레이어가 될 수 있습니다.
            *self.classifier
        ]
        return layers

# PipelineModule로 최종 변환
from deepspeed.pipe import PipelineModule
net = AlexNetPipe()
net = PipelineModule(layers=net.to_layers(), num_stages=2)
```

### 훈련 루프 (Training Loops)

파이프라인 병렬처리는 Forward와 Backward Pass를 겹쳐서(interleave) 실행합니다. 따라서 기존처럼 forward(), backward(), step()을 명시적으로 분리하여 호출할 수 없습니다.

대신, DeepSpeed 엔진은 train_batch()라는 단일 메서드를 제공합니다. 이 메서드는 다음 배치의 데이터가 소모되고 모델 가중치가 업데이트될 때까지 파이프라인을 알아서 실행합니다.

```python
# 기존 Data Parallel 훈련 루프 (참고용)
data_iter = iter(train_loader)
for micro_batch in engine.gradient_accumulation_steps():
    batch = next(data_iter)
    loss = engine(batch)
    engine.backward(loss)
engine.step()

# DeepSpeed Pipeline 훈련 루프 (이것만 쓰면 됨)
train_iter = iter(train_loader)
loss = engine.train_batch(data_iter=train_iter)
```

### 데이터 처리 (Dealing with Data)

파이프라인 환경에서는 첫 번째 스테이지(Stage 0)만 입력 데이터를 사용하고, 마지막 스테이지만 라벨(label)을 사용해 손실(loss)을 계산합니다.

DeepSpeed 엔진은 데이터 로더가 (입력 데이터, 라벨 데이터) 형태의 튜플을 반환할 것으로 기대합니다.

가장 편리한 방법은 deepspeed.initialize()에 training_data 인자로 dataset을 넘겨주는 것입니다. DeepSpeed가 알아서 분산 데이터 로더를 구성해 줍니다. 이 경우 훈련 루프는 훨씬 더 단순해집니다.

```python
# dataset을 initialize에 전달
engine, _, _, _ = deepspeed.initialize(
    args=args,
    model=net,
    model_parameters=[...],
    training_data=cifar_trainset() # 데이터셋을 여기서 전달
)

# 훈련 루프
for step in range(args.steps):
    loss = engine.train_batch() # 데이터 이터레이터를 넘길 필요도 없음
```

## 3. Automatic tensor paralleism (transformers) 

### 서론

이 튜토리얼은 \*\*추론(Inference)\*\*을 위한 새로운 **자동 텐서 병렬처리(Automatic Tensor Parallelism)** 기능을 소개합니다.

이전에는 텐서 병렬처리(TP)를 활성화하기 위해 사용자가 DeepSpeed에 \*\*'주입 정책(injection policy)'\*\*을 직접 제공해야 했습니다. 하지만 이제 DeepSpeed는 커널 주입(kernel injection)이 활성화되지 않고 주입 정책이 제공되지 않은 경우, **HuggingFace 모델에 대해 기본적으로 자동 텐서 병렬처리를 지원**합니다.

이 기능 덕분에, 현재 커널 주입이 지원되지 않는 모델이라도 사용자가 직접 주입 정책을 제공할 필요 없이 성능을 향상시킬 수 있습니다.

**커널 주입이란?** 원래 GPU에서 실행되는 특정 연산(예: 행렬 곱셈)을 최적화된 연산들로 대체하는 것을 의미합니다.
예를 들어... GPU0과 GPU1 이 있을때 VRAM으로 올리고 - GPU0 연산하고, GPU1 연산하고 - 결과를 합치는 식으로 최적화하던 것을 tensor parallelism 에 맞게 일부는 GPU0 에서, 일부는 GPU1 에서 동시에 연산하고 합치고 하는 식으로 직접 선언해서 최적화하는 것을 뜻합니다.


```python
# ---------------------------------------
# 새로운 자동 텐서 병렬처리 방식
# ---------------------------------------
import os
import torch
import transformers
import deepspeed

local_rank = int(os.getenv("LOCAL_RANK", "0"))
world_size = int(os.getenv("WORLD_SIZE", "1"))

# 모델 파이프라인 생성
pipe = transformers.pipeline(task="text2text-generation", model="google/t5-v1_1-small", device=local_rank)

# DeepSpeed-Inference 엔진 초기화
# ★★★ tp_size (mp_size)만 지정하면 자동으로 TP가 적용됩니다 ★★★
pipe.model = deepspeed.init_inference(
    pipe.model,
    mp_size=world_size, # mp_size는 tp_size와 같습니다.
    dtype=torch.float
)

output = pipe('Input String')
```

이전에는 커널 주입이 지원되지 않는 모델에 텐서 병렬처리만 적용하려면, Transformer의 Encoder/Decoder 레이어에 있는 특정 두 Linear 레이어(1. 어텐션 출력 GeMM, 2. 레이어 출력 GeMM)를 명시하는 주입 정책을 전달해야 했습니다. 이는 GPU 간의 All-Reduce 통신을 추가하여 부분적인 결과를 병합하기 위해 필요했습니다.

아래는 **이전 방식**의 예시입니다.

```python
# ----------------------------------
# 이전의 텐서 병렬처리 방식
# ----------------------------------
import os
import torch
import transformers
import deepspeed
from transformers.models.t5.modeling_t5 import T5Block # ★모델 내부 블록 임포트 필요

local_rank = int(os.getenv("LOCAL_RANK", "0"))
world_size = int(os.getenv("WORLD_SIZE", "1"))

# 모델 파이프라인 생성
pipe = transformers.pipeline(task="text2text-generation", model="google/t5-v1_1-small", device=local_rank)

# DeepSpeed-Inference 엔진 초기화
# ★★★ injection_policy를 수동으로 지정해야 했음 ★★★
pipe.model = deepspeed.init_inference(
    pipe.model,
    mp_size=world_size,
    dtype=torch.float,
    injection_policy={T5Block: ('SelfAttention.o', 'EncDecAttention.o', 'DenseReluDense.wo')}
)
output = pipe('Input String')
```

여기서 `injection_policy={T5Block: ('SelfAttention.o', 'EncDecAttention.o', 'DenseReluDense.wo')}` 가 각각 T5Block 내의 Self-Attention 출력, Encoder-Decoder Attention 출력, Feed-Forward 네트워크 출력에 해당하는 Linear 레이어들을 지정합니다. 이 레이어들에 텐서 병렬처리가 적용되며, DeepSpeed는 자동으로 필요한 All-Reduce 연산을 삽입합니다. 즉 여기 선언된 연산들에 들어가는 Tensor들을 GPU로 분할하고, 동시에 연산하고, 합치는 방식의 최적화를 선언합니다.


자동 텐서 병렬처리를 사용하면, 지원되는 모델에 대해 더 이상 주입 정책을 제공할 필요가 없습니다. 주입 정책은 런타임에 결정되어 자동으로 적용됩니다.

-----

### 예제 스크립트

https://github.com/deepspeedai/DeepSpeedExamples/blob/master/inference/huggingface/text-generation/inference-test.py

Inference 테스트 스크립트(`inference-test.py`)를 사용하여 자동 텐서 병렬처리의 성능 향상을 관찰할 수 있습니다. 이 스크립트는 텍스트 생성 모델을 테스트하기 위한 것이며, 비교를 위해 토큰당 지연 시간(latency), 대역폭(bandwidth), 처리량(throughput), 메모리 사용량을 확인하는 기능이 포함되어 있습니다.

#### 스크립트 실행

다음 명령어는 DeepSpeed나 텐서 병렬처리 없이 모델을 실행합니다. (`test_performance` 플래그로 성능 데이터를 수집합니다.)

```bash
deepspeed --num_gpus <num_gpus> DeepSpeedExamples/inference/huggingface/text-generation/inference-test.py \
    --model <model> \
    --batch_size <batch_size> \
    --test_performance
```

텐서 병렬처리를 활성화하려면, 호환되는 모델에 대해 `ds_inference` 플래그를 사용하면 됩니다. (이 플래그가 자동 TP를 활성화합니다.)

```bash
deepspeed --num_gpus <num_gpus> DeepSpeedExamples/inference/huggingface/text-generation/inference-test.py \
    --model <model> \
    --batch_size <batch_size> \
    --test_performance \
    --ds_inference
```

---

### 우리 예제

```terminal
# 해당디렉토리로 이동해서 실행
deepspeed --num_gpus 2 inference-test.py --model facebook/opt-125m
# 비교용
deepspeed --num_gpus 1 inference-test.py --model facebook/opt-125m
```



(albert, arctic, baichuan, bert, bloom, codellama, falcon, gpt-j, gpt-neo, gpt-neox, llama, llama2, mistral, mixtral, mpt, opt, phi, qwen, qwen2, t5, xglm 등 다수)

### 지원되지 않는 모델

다음 모델은 현재 자동 텐서 병렬처리를 지원하지 않습니다. (하지만 Bloom의 커널 주입과 같은 다른 DeepSpeed 기능과는 여전히 호환될 수 있습니다.)

(deberta, flaubert, fsmt, gpt2, led, longformer, xlm, xlnet)

## +) ZeRO-offload ZeRO-infinity

GPU 나눠서하는거 좋다.. 근데 VRAM이 여전히 작아버릴 수도 있잖아요? 그때 RAM과 NVMe(대충 빠르다는 뜻) SSD까지 사용합니다.

### 1\. The Problem: 왜 ZeRO-3로도 부족한가?

ZeRO-3는 모델 가중치(Parameters)를 모든 GPU에 쪼개서(shard) 저장합니다. 하지만 여전히 **3가지**의 큰 메모리 소비 요소가 GPU VRAM을 차지합니다.

1.  **모델 가중치 (Parameters):** 1T 모델이면 1T \* 2(fp16) = 2TB. 8개 GPU로 쪼개도 GPU당 256GB. (불가능)
2.  **그래디언트 (Gradients):** 훈련 중 발생하는 기울기 (가중치와 크기 동일)
3.  **옵티마이저 상태 (Optimizer States):** Adam 옵티마이저의 경우, 가중치의 **12배\~16배** 크기. (가장 큰 범인)

ZeRO-3가 아무리 잘게 쪼개도, GPU VRAM(예: 80GB)은 이들을 감당할 수 없습니다.

### 2\. The Solution: ZeRO-Offload (CPU로의 위임)

ZeRO-Offload의 핵심 아이디어는 간단합니다.

> "GPU는 **계산**에만 집중하고, **저장**은 VRAM보다 훨씬 용량이 큰 **CPU RAM**에 맡기자."

GPU는 당장 계산에 필요한 데이터만 CPU RAM에서 가져오고, 계산이 끝나면 다시 CPU RAM으로 돌려보냅니다.

  * **무엇을 Offload 하는가?**
      * **옵티마이저 상태 (`offload_optimizer`):** 가장 크고, `optimizer.step()` 순간에만 잠깐 필요합니다. Offload 1순위이며 효과가 가장 좋습니다. (ZeRO-2와도 호환됩니다)
      * **모델 가중치 (`offload_param`):** ZeRO-3와 함께 사용되며, 모델 가중치 자체도 CPU RAM에 둡니다.

-----

### 3\. `ds_config.json` 예시

강의에서 이 JSON 설정의 변화를 보여주는 것이 핵심입니다.

#### 예시 1: ZeRO-2 + 옵티마이저 Offload (가장 실용적인 조합)

VRAM은 부족하지만 CPU RAM은 넉넉할 때 최고의 선택입니다.

```json
{
  "zero_optimization": {
    "stage": 2,
    
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    }
  },
  "train_micro_batch_size_per_gpu": 16
}
```

  * `"device": "cpu"`: 옵티마이저 상태(Adam의 모멘텀 등)를 VRAM이 아닌 CPU RAM에 저장합니다.

#### 예시 2: ZeRO-3 + 파라미터 & 옵티마이저 Offload (초거대 모델)

모델 자체가 너무 커서 VRAM에 쪼개서도 못 올릴 때 사용합니다.

```json
{
  "zero_optimization": {
    "stage": 3,
    
    "offload_param": {
      "device": "cpu",
      "pin_memory": true
    },

    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    }
  },
  "train_micro_batch_size_per_gpu": 16
}
```

  * `"offload_param"`: ZeRO-3로 쪼갠 모델 가중치마저 CPU RAM에 둡니다.

-----

### 4\. ZeRO-Infinity (NVMe SSD 활용)

**"CPU RAM (예: 512GB)으로도 감당이 안 되는 수십 TB급 모델은 어떡할까요?"**

이것이 **ZeRO-Infinity**가 해결하는 문제입니다.

ZeRO-Infinity는 메모리 계층을 VRAM, CPU RAM에서 한 단계 더 확장하여, 용량이 TB 단위인 **NVMe SSD**까지 저장소로 활용합니다.

  * **동작 원리:** 3-Tier 메모리 관리

    1.  **VRAM (GPU):** 현재 계산에 필요한 데이터만 위치 (가장 빠름)
    2.  **CPU RAM:** VRAM에서 밀려난 데이터 + 곧 필요할 데이터 (중간)
    3.  **NVMe SSD:** 당장 필요 없는 거대한 모델/옵티마이저 상태 (가장 큼, 가장 느림)

  * DeepSpeed는 '사전 페치(pre-fetching)' 기술을 사용해, GPU가 특정 레이어를 계산하기 *직전에* NVMe -\> CPU RAM -\> VRAM으로 데이터를 미리 불러와서 느린 속도를 최대한 숨깁니다.

#### `ds_config.json` (NVMe 활성화 예시)

`"device"`를 `"cpu"` 대신 `"nvme"`로 바꿉니다.

```json
{
  "zero_optimization": {
    "stage": 3,
    
    "offload_param": {
      "device": "nvme",
      "pin_memory": true,
      "nvme_path": "/path/to/my/fast_nvme_storage" 
    },

    "offload_optimizer": {
      "device": "nvme",
      "pin_memory": true,
      "nvme_path": "/path/to/my/fast_nvme_storage"
    }
  },
  "train_micro_batch_size_per_gpu": 16
}
```

  * `"device": "nvme"`: 데이터를 CPU RAM이 아닌 NVMe SSD에 저장합니다.
  * `"nvme_path"`: 데이터를 저장할 NVMe 마운트 경로를 지정해야 합니다.


* **ZeRO-Offload (CPU):** VRAM의 한계를 **CPU RAM**으로 확장합니다. GPU가 계산하는 동안 CPU는 데이터 전송(PCIe)을 수행하여 병목을 숨깁니다.
* **ZeRO-Infinity (NVMe):** CPU RAM의 한계마저 **NVMe SSD**로 확장합니다. 1조(Trillion) 파라미터가 넘는, 사실상 '무한한' 크기의 모델을 훈련할 수 있게 해주는 최종 기술입니다.




## +) Data Efficiency

https://www.deepspeed.ai/docs/config-json/#data-efficiency

보통 GPU는 연산을 무진장 많이 하니까 CPU -> GPU 로 데이터 로딩이 병목현상이 되는 경우가 많습니다. 이걸 그냥 기다리고 있으면 GPU가 놀게 되니까, DeepSpeed에서는 데이터 로딩을 더 효율적으로 하기 위한 몇가지 옵션들을 제공합니다. json 에서 설정을 해줄 수 있습니다. 자세한건 저도 잘 모르고 매번 업데이트 되어서 구현된 기능이 변경되곤 합니다.